Just for testing:

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-03 06:21:43.454209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 06:21:45.191670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import numpy as np
import pandas as pd
import re
import spacy

nlp = spacy.load('en_core_web_sm')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path_name = "/content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_"
link = path_name + str(1) + '.txt'

In [5]:
def read_jeopardy_file(filename):
  regex = r'^\[\[.*?\]\]\n'

  # Define a regular expression pattern to match non-alphabetic characters
  pattern = re.compile(r'[^a-zA-Z0-9 ]+')

  with open(filename, "r", encoding="utf8") as f:
    text = f.read()
  documents = re.split(regex, text)[1:]
  objects = []
  
  title_of_each_page = text.splitlines()[0]
  title_of_each_page = title_of_each_page.replace('[[', '').replace(']]', '').strip()
  next_title = ''

  for document in documents:
    # Split the document into a list of lines
    lines = document.strip().split('\n')

    # Initialize an empty string to store the object
    obj = ''
    
    # Loop through each line in the document
    for line in lines:
      # Check if the line starts with [[ and ends with ]]
      if re.match(r'^\[\[.*?\]\]$', line):
        next_title = line.replace('[[', '').replace(']]', '').strip()
        # If the object is not empty, add it to the list of objects
        if obj:
          tokens = obj.split()
          new_token_list = []
          for each in tokens:
            # TODO: more filtering in tokens
            # www, ....
            if ('url' in each):
                continue
            elif ('http' in each):
                continue
            else:
                new_token_list.append(each)
          title_of_each_page = re.sub(pattern, ' ', title_of_each_page)
          title_of_each_page = ' '.join(title_of_each_page.split())
          current_page_info = {'title': title_of_each_page.lower(), 'doc_index': 0, 'tokens': new_token_list, 'ner_tags_str': []}
          #objects.append(obj.strip())
          objects.append(current_page_info)
          # store with title
          title_of_each_page = next_title

        # Reset the object
        obj = ''
      else:
        # Add the line to the object
        obj += line + ' '
    # If there is an object at the end of the document, add it to the list of objects
    if obj:
      tokens = obj.split()
      title_of_each_page = re.sub(pattern, ' ', title_of_each_page)
      title_of_each_page = ' '.join(title_of_each_page.split())
      current_page_info = {'title': title_of_each_page.lower(), 'doc_index': 0, 'tokens': tokens, 'ner_tags_str': []}
      objects.append(current_page_info)
    
    return objects

In [6]:
results = read_jeopardy_file(link)

In [7]:
print(len(results))

print(results[0])
print(results[1])
print(results[2])

649
{'title': 'bbs', 'doc_index': 0, 'tokens': ['BBS', 'may', 'refer', 'to:', '==Technologies==', '==Organisations==', '==Science==', '==Culture==', '==Other=='], 'ner_tags_str': []}
{'title': 'balfour declaration', 'doc_index': 0, 'tokens': ['#REDIRECT', 'Balfour', 'Declaration', '[tpl]R', 'from', 'other', 'capitalisation[/tpl]'], 'ner_tags_str': []}
{'title': 'british standards', 'doc_index': 0, 'tokens': ['CATEGORIES:', '1901', 'establishments', 'in', 'the', 'United', 'Kingdom,', 'British', 'Standards,', 'International', 'Electrotechnical', 'Commission,', 'Standards', 'organizations,', 'Certification', 'marks,', 'Organisations', 'based', 'in', 'the', 'United', 'Kingdom', 'British', 'Standards', 'are', 'the', 'standards', 'produced', 'by', 'BSI', 'Group', 'which', 'is', 'incorporated', 'under', 'a', 'Royal', 'Charter', '(and', 'which', 'is', 'formally', 'designated', 'as', 'the', 'National', 'Standards', 'Body', '(NSB)', 'for', 'the', 'UK).', 'The', 'BSI', 'Group', 'produces', 'Briti

In [8]:
full_doc_list = []
token_dict_per_doc = []
counter = 0

# call the read_jeopardy_file() function for each file in the wiki_dataset folder
for i in range(80):
    # pass the filename to the function
    each_filename = path_name + str(i + 1) + '.txt'
    
    page_list = read_jeopardy_file(each_filename)

    print(len(page_list), ' ', each_filename)
    
    for each_item in page_list:
        each_item['doc_index'] = counter
        full_doc_list.append(each_item)
        counter = counter + 1

649   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_1.txt
634   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_2.txt
728   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_3.txt
700   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_4.txt
552   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_5.txt
601   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_6.txt
577   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_7.txt
808   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_8.txt
724   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_9.txt
697   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_10.txt
674   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_11.txt
728   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_12.txt
516   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_13.txt
628   /content/drive/MyDrive/583_TR_Project/wiki_dataset/doc_14.txt
571   /content/drive/MyDrive/583_TR_Project/wiki_dataset/

In [9]:
print(full_doc_list[0])
print(full_doc_list[1])

print(full_doc_list[279823])

{'title': 'bbs', 'doc_index': 0, 'tokens': ['BBS', 'may', 'refer', 'to:', '==Technologies==', '==Organisations==', '==Science==', '==Culture==', '==Other=='], 'ner_tags_str': []}
{'title': 'balfour declaration', 'doc_index': 1, 'tokens': ['#REDIRECT', 'Balfour', 'Declaration', '[tpl]R', 'from', 'other', 'capitalisation[/tpl]'], 'ner_tags_str': []}
{'title': 'google test', 'doc_index': 279823, 'tokens': ['CATEGORIES:', 'C++', 'libraries,', 'Extreme', 'programming,', 'Freedesktop.org,', 'Unit', 'testing', 'frameworks', 'Google', 'Test', 'is', 'a', 'unit', 'testing', 'library', 'for', 'the', 'C++', 'programming', 'language,', 'based', 'on', 'the', 'xUnit', 'architecture.A', 'quick', 'introduction', 'to', 'the', 'Google', 'C++', 'Testing', 'Framework,', 'Arpan', 'Sen,', 'IBM', 'DeveloperWorks,', '2010-05-11,', 'retrieved', '2013-08-14', 'The', 'library', 'is', 'released', 'under', 'the', 'BSD', '3-clause', 'license.Google', "Test's", 'site,', 'retrieved', '2013-08-14,', 'cites', 'New', 'BS

In [10]:
print('There are ', len(full_doc_list), ' pages in those 80 files')

There are  279824  pages in those 80 files


In [58]:
for each in full_doc_list:
  #print(each['title'])
  
  #if ('al-quds' in each['title']): #al-quds al-arabi
  #if ('sun sen' in each['title']): #sun-sentinel
  #if ('morning star' in each['title']): #morning star (british newspaper)
  #if ('the atlanta' in each['title']): #the atlanta journal-constitution
  #if ('sun ' in each['title']): #possible error
  #if ('united nations security' in each['title']): #sun-sentinel
  if ('hare international airport' in each['title']): #O'Hare International Airport
    print(each)
    #break

{'title': 'o hare international airport', 'doc_index': 46447, 'tokens': ['category', 'o', 'hare', 'international', 'airport', 'airfield', 'of', 'the', 'united', 'states', 'army', 'air', 'forces', 'in', 'illinois', 'airfield', 'of', 'the', 'united', 'states', 'army', 'air', 'forces', 'technical', 'service', 'command', 'airports', 'establish', 'in', '1943', 'airport', 'in', 'cook', 'county', 'illinois', 'airports', 'in', 'dupage', 'county', 'illinois', 'american', 'airlines', 'buildings', 'and', 'structure', 'in', 'chicago', 'illinois', 'united', 'airlines', 'chicago', 'o', 'hare', 'international', 'airport', 'tpl', 'airport', 'codes', 'ord', 'kord', 'ord', 'tpl', 'also', 'know', 'as', 'o', 'hare', 'airport', 'o', 'hare', 'field', 'chicago', 'international', 'airport', 'or', 'simply', 'o', 'hare', 'be', 'a', 'major', 'airport', 'locate', 'in', 'the', 'northwestern', 'most', 'corner', 'of', 'chicago', 'illinois', 'united', 'states', 'tpl', 'convert', '17', 'mi', 'km', '0', 'tpl', 'northwe

In [12]:
path_name = "/content/drive/MyDrive/583_TR_Project/questions/Q"

In [13]:
# k = 20
def read_top_k_candidates_for_each_question(filename):
  with open(filename, "r", encoding="utf8") as f:
    
    list_of_titles_for_each_question = []

    # Loop through each line in the document
    for line in f:
      line = line.strip()
      list_of_titles_for_each_question.append(line)
    
    return list_of_titles_for_each_question

In [15]:
def retrieve_contents_from_title(title_name):
  for each in full_doc_list:
    if (each['title'] == title_name.strip()):
      #print('found');
      return each

  print('not found:', title_name);
  return 0;

In [17]:
# read 1st one - for testing
link = path_name + str(1) + '/Q' + str(1) + '_candidate_answers'
link = path_name + str(7) + '/Q' + str(7) + '_candidate_answers'
print(link)

first = read_top_k_candidates_for_each_question(link)
print(first)
print('len:', len(first))

documents = []
for each in first:
  each_title = ' '.join(each.split())
  result = retrieve_contents_from_title(each_title)
  documents.append(result)

/content/drive/MyDrive/583_TR_Project/questions/Q7/Q7_candidate_answers
['tintoretto', 'nicolas poussin', 'diego vel zquez', 'giorgio vasari', 'leonardo da vinci', 'carlo dolci', 'icon', 'antonio da correggio', 'caravaggio', 'taddeo zuccari', 'pieter codde', 'lorenzo monaco', 'giovanni battista gaulli', 'nicola cianfanelli', 'list of works by el greco', 'cimabue', 'hieronymus bosch', 'uffizi', 'ardhanarishvara', 'master of the khanenko adoration']
len: 20


In [21]:
documents_list_2000 = []
# call the read_top_k_candidates_for_each_question() function for each questions from folder
for i in range(100):
    i = i + 1
    # pass the filename to the function
    each_filename = path_name + str(i) + '/Q' + str(i) + '_candidate_answers'
    #print(each_filename, '\n')

    list_of_candidates = read_top_k_candidates_for_each_question(each_filename)
    #print(len(list_of_candidates))
    for each in list_of_candidates:
      documents_list_2000.append(each)

    for each_title in list_of_candidates:
      #search if hit or not
      each_title = ' '.join(each_title.split())
      #result = retrieve_contents_from_title(each_title)
      #print(result)
    
    #print('\n\n')

In [20]:
print(documents_list_2000)
print(len(documents_list_2000))

['yellow journalism', 'the new york sun', 'media of the united states', 'william randolph hearst', 'star tribune', 'the age', 'al quds al arabi', 'irish independent', 'new york press', 'los angeles times', 'the spectator', 'sun sentinel', 'morning star  british newspaper', 'houston chronicle', 'reforma', 'the atlanta journal constitution', 'the wall street journal', 'history of british newspapers', 'harte hanks', 'haaretz', 'history of the republic of china', 'one china policy', 'kinmen', 'taiwanization', 'taiwan', 'political status of taiwan', 'korean war', 'united nations security council resolution 1441', 'curtis lemay', 'iraq disarmament crisis', 'united nations general assembly resolution 505', 'lockheed u 2', 'nancy pelosi', 'dennis kucinich', 'republic of china presidential election  2004', 'madagascar', 'rikers island', 'history of oceania', 'history of cuba', 'lin sen', 'the wall street journal', 'true crime', 'the atlanta journal constitution', 'barry minkow', 'michael milken

In [31]:
updated_list = set([' '.join(elem.split()) for elem in documents_list_2000])

In [32]:
print(updated_list)
print(len(updated_list))

{'wesley wyndam pryce', 'french quarter', 'being erica', 'the new york sun', 'cung le', 'louis x of france', 'list of minor the hitchhiker s guide to the galaxy characters', 'santa s little helper', 'khwaja ahmad abbas', 'perkin warbeck', 'panjab university chandigarh', 'maxwell caulfield', 'claudia winkleman', 'list of pepsi spokespersons', 'starlight express', 'second continental congress', 'aaron burr', 'ride the lightning', 'carlo dolci', 'change ringing', 'townshend acts', 'walt whitman', 'bellarmine jefferson high school', 'magnavox odyssey', 'christine brennan', 'wall street 1987 film', 'michael milken', 'terah', 'solomon p sharp', 'jackie mudie', 'ferdinand iv of castile', 'big bounce', 'daniel d tompkins', 'st george jackson mivart', 'huedin', 'mark mckinnon', 'python reticulatus', 'history of albania', 'utah museum of fine arts', 'piura region', 'b nai b rith', 'port vila', 'prime minister of france', 'meaning of life', 'cherokee language', 'breakfast', 'jane taylor musician'

In [34]:
final_set_of_doc_dicts = []
for each_title in updated_list:
  result = retrieve_contents_from_title(each_title)
  final_set_of_doc_dicts.append(result)

In [38]:
print(final_set_of_doc_dicts[0])
print(final_set_of_doc_dicts[1])
print(final_set_of_doc_dicts[1837])

print(len(final_set_of_doc_dicts))

{'title': 'wesley wyndam pryce', 'doc_index': 44778, 'tokens': ['CATEGORIES:', 'Buffy', 'the', 'Vampire', 'Slayer', 'characters,', 'Angel', '(TV', 'series)', 'characters,', 'Fictional', 'characters', 'introduced', 'in', '1999,', 'Fictional', 'English', 'people,', 'Fictional', 'ghosts,', 'Fictional', 'people', 'from', 'London,', 'Fictional', 'private', 'investigators,', 'Fictional', 'vampire', 'hunters,', 'Fictional', 'demon', 'hunters,', 'Fictional', 'members', 'of', 'secret', 'societies,', 'Fictional', 'immigrants', 'to', 'the', 'United', 'States', 'Wesley', 'Wyndam-Pryce', '(also', 'spelled', 'Wyndam-Price', 'and', 'Wyndham-Price)', 'is', 'a', 'fictional', 'character', 'created', 'by', 'Joss', 'Whedon', 'for', 'the', 'television', 'series', 'Buffy', 'the', 'Vampire', 'Slayer', 'and', 'Angel.', 'Played', 'by', 'Alexis', 'Denisof,', 'Wesley', 'first', 'appeared', 'in', 'nine', 'episodes', 'of', 'Buffys', 'third', 'season', 'in', '1999', 'before', 'moving', 'over', 'to', 'spin-off', 'se

===

Apply Lemmatization to only these 2000 (actually 1838) documents

===

In [56]:
# Define a regular expression pattern to match non-alphabetic characters
pattern = re.compile(r'[^a-zA-Z0-9 ]+')

for each in final_set_of_doc_dicts:
  body_text = ' '.join(each['tokens'])
  doc = nlp(body_text)
  lemmatized_tokens = ' '.join([token.lemma_ for token in doc])
  lemmatized_tokens = re.sub(pattern, ' ', lemmatized_tokens)
  lemmatized_tokens = ' '.join(lemmatized_tokens.split()).lower()
  each['tokens'] = lemmatized_tokens.split()
  #print(lemmatized_tokens)
  #break


SyntaxError: ignored

In [43]:
print(final_set_of_doc_dicts[0])
print(final_set_of_doc_dicts[1])
print(final_set_of_doc_dicts[1837])

print(len(final_set_of_doc_dicts))

{'title': 'wesley wyndam pryce', 'doc_index': 44778, 'tokens': ['category', 'buffy', 'the', 'vampire', 'slayer', 'character', 'angel', 'tv', 'series', 'character', 'fictional', 'character', 'introduce', 'in', '1999', 'fictional', 'english', 'people', 'fictional', 'ghost', 'fictional', 'people', 'from', 'london', 'fictional', 'private', 'investigator', 'fictional', 'vampire', 'hunter', 'fictional', 'demon', 'hunter', 'fictional', 'member', 'of', 'secret', 'society', 'fictional', 'immigrant', 'to', 'the', 'united', 'states', 'wesley', 'wyndam', 'pryce', 'also', 'spell', 'wyndam', 'price', 'and', 'wyndham', 'price', 'be', 'a', 'fictional', 'character', 'create', 'by', 'joss', 'whedon', 'for', 'the', 'television', 'series', 'buffy', 'the', 'vampire', 'slayer', 'and', 'angel', 'play', 'by', 'alexis', 'denisof', 'wesley', 'first', 'appear', 'in', 'nine', 'episode', 'of', 'buffys', 'third', 'season', 'in', '1999', 'before', 'move', 'over', 'to', 'spin', 'off', 'series', 'angel', 'where', 'he'

In [44]:
import pickle

In [45]:
file_name = "/content/drive/MyDrive/583_TR_Project/my_pickle_file.pkl"
# Open a file to write the pickle data
with open(file_name, 'wb') as f:
    # Dump the list of dictionaries to the file using pickle.dump()
    pickle.dump(final_set_of_doc_dicts, f)

In [50]:
# Open the pickle file in read mode
with open(file_name, 'rb') as f:
    # Load the serialized data from the file using pickle.load()
    final_set_of_doc_dicts = pickle.load(f)

# Print the loaded list of dictionaries
print(final_set_of_doc_dicts[0])
print(final_set_of_doc_dicts[1])
print(final_set_of_doc_dicts[1837])

print(len(final_set_of_doc_dicts))

{'title': 'wesley wyndam pryce', 'doc_index': 44778, 'tokens': ['category', 'buffy', 'the', 'vampire', 'slayer', 'character', 'angel', 'tv', 'series', 'character', 'fictional', 'character', 'introduce', 'in', '1999', 'fictional', 'english', 'people', 'fictional', 'ghost', 'fictional', 'people', 'from', 'london', 'fictional', 'private', 'investigator', 'fictional', 'vampire', 'hunter', 'fictional', 'demon', 'hunter', 'fictional', 'member', 'of', 'secret', 'society', 'fictional', 'immigrant', 'to', 'the', 'united', 'states', 'wesley', 'wyndam', 'pryce', 'also', 'spell', 'wyndam', 'price', 'and', 'wyndham', 'price', 'be', 'a', 'fictional', 'character', 'create', 'by', 'joss', 'whedon', 'for', 'the', 'television', 'series', 'buffy', 'the', 'vampire', 'slayer', 'and', 'angel', 'play', 'by', 'alexis', 'denisof', 'wesley', 'first', 'appear', 'in', 'nine', 'episode', 'of', 'buffys', 'third', 'season', 'in', '1999', 'before', 'move', 'over', 'to', 'spin', 'off', 'series', 'angel', 'where', 'he'

= = = 

Retrieve all questions from questions.txt

Make a List of dictionary: 
[{'category': 'NEWSPAPERS', 'question': "The dominant paper in our nation's capital, it's among the top 10 U.S. papers in circulation", 'answer': 'The Washington Post'}, 
.....
]

= = = 

In [51]:
path_name = "/content/drive/MyDrive/583_TR_Project/questions.txt"

In [52]:
# Open the file for reading
with open(path_name, "r", encoding="utf8") as f:
    # Initialize variables
    objects = []
    current_obj = {}

    # Loop over each line in the file
    for line in f:
        # Strip any whitespace and skip empty lines
        line = line.strip()
        if not line:
            continue

        # Check if this line is a category
        if not current_obj:
            current_obj["category"] = line
        else:
            # Add the question or answer to the current object
            if "question" not in current_obj:
                current_obj["question"] = line
            else:
                current_obj["answer"] = line
                # Add the current object to the list and reset it
                objects.append(current_obj)
                current_obj = {}

    # Add the last object to the list
    if current_obj:
        objects.append(current_obj)

# Print the list of objects
for obj in objects:
    print(obj)


{'category': 'NEWSPAPERS', 'question': "The dominant paper in our nation's capital, it's among the top 10 U.S. papers in circulation", 'answer': 'The Washington Post'}
{'category': "OLD YEAR'S RESOLUTIONS", 'question': 'The practice of pre-authorizing presidential use of force dates to a 1955 resolution re: this island near mainland China', 'answer': 'Taiwan'}
{'category': 'NEWSPAPERS', 'question': 'Daniel Hertzberg & James B. Stewart of this paper shared a 1988 Pulitzer for their stories about insider trading', 'answer': 'The Wall Street Journal'}
{'category': 'BROADWAY LYRICS', 'question': 'Song that says, "you make me smile with my heart; your looks are laughable, unphotographable"', 'answer': 'My Funny Valentine'}
{'category': 'POTPOURRI', 'question': 'In 2011 bell ringers for this charity started accepting digital donations to its red kettle', 'answer': 'The Salvation Army|Salvation Army'}
{'category': "STATE OF THE ART MUSEUM (Alex: We'll give you the museum. You give us the stat

= = = 

TF-IDF score of a query string

===

In [54]:
#for each in final_set_of_doc_dicts:
#  print(each)
print('len: ', len(final_set_of_doc_dict))

len:  1838


In [55]:
import math

# Define list of documents (each document is a dictionary with 'tokens' key)
documents = final_set_of_doc_dicts  # Replace [...] with your list of documents
print(len(documents))

# Define query string
#query = "your query string"
query = objects[6]['question']
print(query)

# Compute document frequency (DF) for each term in the query
df = {}
for doc in documents:
    for token in set(doc['tokens']):
        if token in df:
            df[token] += 1
        else:
            df[token] = 1

# Compute inverse document frequency (IDF) for each term in the query
N = len(documents)
idf = {}
for token in df:
    idf[token] = math.log(N / df[token])

# Compute term frequency (TF) for each term in the query for each document
tf = {}
for doc_idx, doc in enumerate(documents):
    tf[doc_idx] = {}
    for token in set(doc['tokens']):
        tf[doc_idx][token] = doc['tokens'].count(token) / len(doc['tokens'])

# Compute TF-IDF score for each term in the query for each document
tfidf = {}
for doc_idx, doc in enumerate(documents):
    tfidf[doc_idx] = 0
    for token in query.split():
        if token in tf[doc_idx]:
            tfidf[doc_idx] += tf[doc_idx][token] * idf[token]

# Print TF-IDF scores for each document
for doc_idx, score in sorted(tfidf.items(), key=lambda x: x[1], reverse=True):
    print(f"Document {doc_idx}: {score}")


1838
This Italian painter depicted the "Adoration of the Golden Calf"
Document 1436: 0.0730027719751744
Document 302: 0.04505519452691015
Document 843: 0.04347808733754596
Document 1119: 0.03607916022702532
Document 1080: 0.0330121767914824
Document 872: 0.03272338644595506
Document 454: 0.029564827905812788
Document 18: 0.02215720035001596
Document 1581: 0.020138369678894704
Document 104: 0.02000117458048803
Document 863: 0.01945864290059351
Document 399: 0.01922464424132547
Document 242: 0.018851881757706704
Document 1097: 0.017942068860244454
Document 1068: 0.017621110351532403
Document 1376: 0.01737621458930349
Document 1280: 0.014412819526398214
Document 1255: 0.014333030386871426
Document 1060: 0.01425248845031965
Document 987: 0.01363636143188975
Document 1779: 0.011253580328498809
Document 163: 0.011182135449928871
Document 1547: 0.011072837245750438
Document 223: 0.010169198733244124
Document 271: 0.008102697676268434
Document 1075: 0.007891900684065119
Document 949: 0.0073913